# Manipulation.py: Pushing example


This tutorial provides an example of how to plan a pushing motion on a block using the manipulation.py helper functions

As always we start with the necessary imports

In [1]:
import numpy as np
import robotic as ry
import manipulation as manip

Now we load a predefined configuration with a robot arm and add a red box

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

midpoint = np.array([-0.105, 0.4, 0.705-.025+.1])
C.addFrame("box") \
    .setPosition(midpoint) \
    .setShape(ry.ST.ssBox, size=[0.08, 0.12, 0.08, 0.001]) \
    .setColor([1, 0, 0]) \
    .setContact(1) \
    .setMass(.1)

# For convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

## Defining the push motion

Using the ManipullationModelling class we can firstly generate two key time slices, one represents the joint state at the start of the push motion in front of the box and another at the end target position of the box. From that we ca fill in the sub-motions, states in between the two key frames, with a few constraints. For the first sub-motion we want the robot to retract the push, this means moving away from teh object before we approach the object leaving us in the desired starting position for our push.

For the second sub-motion we initially ignore all collisions before defining our own explicit collision pairs, these being the robot's fingers with the table and the robot's hand with the table. We have to do this manually since the pushing motion requires us to collide against the box, otherwise we would not find any feasible motions.

In [3]:
def push(obj: str, target_pos: list[float], info: str="", vis: bool=False) -> tuple:
	M = manip.ManipulationModelling(C, f"push_motion{info}", ['l_gripper'])
	M.setup_pick_and_place_waypoints(gripper, obj, 1e-1)
	M.straight_push([1., 2.], obj, gripper, table)
	M.target_xy_position([2.], obj, target_pos)
	M.solve()
	if not M.feasible:
		return False, []

	M1 = M.sub_motion(0)
	M1.retractPush([.0, .15], gripper, .03)
	M1.approachPush([.85, 1.], gripper, .03)
	path1 = M1.solve()
	if not M1.feasible:
		return False, []

	M2 = M.sub_motion(1, accumulated_collisions=False)
	M2.komo.addObjective([], ry.FS.positionRel, [gripper, '_push_start'], ry.OT.eq, 1e1*np.array([[1,0,0],[0,0,1]]))
	M2.keep_distance([], "l_finger1", "table")
	M2.keep_distance([], "l_finger2", "table")
	M2.keep_distance([], "l_palm", "table")
	path2 = M2.solve()
	if not M2.feasible:
		return False, []
	
	if vis:
		M1.play(C, 1.)
		C.attach(gripper, obj)
		M2.play(C, 1.)
		C.attach(table, obj)
	
	path = np.append(path1, path2, 0)
	return True, path

## Visualizing and Executing the motion

We will want to test the output of our function with a real robot, or at least a physics simulation. For this porpuse we can define a bot object with which we will be able to interface with the robot.

In [4]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)

We will attempt to perform 30 push motions. For each successful motion we will first loop through all the calculated joint states in each path, attaching the box to the robot's gripper in the second sub-motion. ManipulationModelling has a convenient function for this called "play", which can be run once the optimizer has found a feasible path. After visualizing the motion in the configuration, we can test it in the simulation (or real robot).

In [5]:
attempt_count = 30
success_count = 0

for l in range(attempt_count):

	target_position = [
		midpoint[0] + np.random.uniform(-.15, .15),
		midpoint[1] + np.random.uniform(-.15, .15),
		0.]
	
	success, path = push(box, target_position, str(l), vis=True)

	if success:
		bot.move(path, [3])
		while bot.getTimeToEnd() > 0.:
			bot.sync(C)
		
	success_count += 1 if success else 0
	
print(f"Successful motions: {success_count}/{attempt_count}")

  -- feasible:push_motion0
     { time: 0.028632, evals: 55, done: 1, feasible: 1, sos: 0.27315, f: 0, ineq: 0, eq: 0.0889668 }
  -- feasible:sub_motion_0--push_motion0
     { time: 0.187483, evals: 93, done: 1, feasible: 1, sos: 2.90589, f: 0, ineq: 1.18567e-06, eq: 0.107973 }
  -- feasible:sub_motion_1--push_motion0
     { time: 0.025982, evals: 11, done: 1, feasible: 1, sos: 0.0515292, f: 0, ineq: 0, eq: 0.0204888 }
  -- feasible:push_motion1
     { time: 0.075368, evals: 200, done: 1, feasible: 1, sos: 0.30828, f: 0, ineq: 0, eq: 0.0476695 }
  -- feasible:sub_motion_0--push_motion1
     { time: 0.475977, evals: 201, done: 1, feasible: 1, sos: 3.45392, f: 0, ineq: 0.00304668, eq: 0.104026 }
  -- feasible:sub_motion_1--push_motion1
     { time: 0.024555, evals: 11, done: 1, feasible: 1, sos: 0.431257, f: 0, ineq: 0, eq: 0.00554877 }
  -- feasible:push_motion2
     { time: 0.069844, evals: 166, done: 1, feasible: 1, sos: 0.468526, f: 0, ineq: 0, eq: 0.0806799 }
  -- feasible:sub_motio